### Glossário do Projeto Delivery_Unit_Economics

Este glossário define os termos de negócio e os componentes técnicos usados no pipeline de ETL (Extract, Transform, Load) no Databricks.

#### 1. Termos de Negócio e Métricas Financeiras
Estes termos são cruciais para a análise de Profit & Loss (P&L) e Unit Economics.

| Termo | Definição |
| :--- | :--- |
| Marketplace Two-Sided | Modelo de negócio da plataforma, que conecta dois grupos distintos e interdependentes (Lojas/Fornecedores e Motoristas/Logística) aos Clientes. |
| Unit Economics (UE) | Análise da rentabilidade de uma única transação (o pedido). O objetivo final do pipeline é calcular o Lucro Bruto Unitário para determinar se o pedido é financeiramente viável. |
| P&L (Profit & Loss) Unitário | Análise da Demonstração de Resultados (Receitas e Custos) focada no nível do pedido. Utilizamos esta análise para determinar a Margem de Contribuição da plataforma por transação. |
| GMV (Gross Merchandise Value) | Valor Bruto da Mercadoria. A receita total gerada por um pedido, incluindo o subtotal do produto e a taxa de entrega (subtotal_bruto + delivery_fee_cliente). |
| COGS (Cost of Goods Sold) | Custo dos Produtos Vendidos. No nosso contexto, refere-se aos Custos Variáveis diretos da transação: o Custo Logístico (Repasse ao Motorista) e o Custo Transacional (Taxa de pagamento). |
| Lucro Bruto Unitário | A métrica final do pipeline. Representa a Margem de Contribuição da plataforma em cada pedido após subtrair os COGS da Receita Líquida. |
| Receita Líquida Marketplace | Valor efetivamente recebido pela plataforma após dedução das taxas (commission_fee + delivery_fee). Fórmula: (commission_fee + delivery_fee) |
| Custo Logístico | Repasse total ao motorista (delivery_fee_driver + bonus_fee_driver) |
| Custo Transacional | Taxa de processamento de pagamento (payment_fee) sobre o GMV |
| Margem de Contribuição | Sinônimo de Lucro Bruto Unitário no contexto do projeto |
| **ROI (Return on Investment)** | Retorno sobre o Investimento. No contexto do pipeline, mede o retorno financeiro da otimização incremental (Economia/Custo de desenvolvimento) |
| **DBU (Databricks Unit)** | Unidade de consumo computacional no Databricks, utilizada para cálculo de custos do pipeline |

#### 2. Fórmulas Implementadas no Pipeline

| Métrica | Fórmula |
| :--- | :--- |
| GMV | `subtotal_bruto + delivery_fee_cliente` |
| Receita Líquida | `commission_fee + delivery_fee` |
| Custo Logístico | `delivery_fee_driver + bonus_fee_driver` |
| Custo Transacional | `payment_fee` |
| Lucro Bruto Unitário | `receita_liquida - custo_logistico - custo_transacional` |
| **ROI Otimização** | `(Economia Anual - Custo Desenvolvimento) / Custo Desenvolvimento * 100` |

#### 3. Termos de Engenharia de Dados e Arquitetura

Estes termos descrevem como o pipeline de dados está estruturado no Databricks/Delta Lake.

| Termo | Definição |
| :--- | :--- |
| Arquitetura de Medalhões | Modelo de pipeline de dados (Bronze → Silver → Gold) usado para garantir qualidade, governança e confiabilidade dos dados à medida que eles progridem e são transformados. |
| Delta Lake | Tecnologia subjacente para todas as tabelas, garantindo ACID properties e versionamento |
| PySpark | Framework utilizado para todas as transformações distribuídas |
| Star Schema | Modelo implementado com: <br>- **Fato**: `tbl_fato_delivery_gold`<br>- **Dimensões**: `tbl_dim_store_gold`, `tbl_dim_time_gold` |
| Chave de Tempo | Gerada a partir de `TO_DATE(order_created_at)` na camada Gold |
| **Incremental Loading** | Estratégia de processamento onde apenas dados novos/modificados são processados, reduzindo custo em 80-90% comparado ao full refresh |
| **Full Refresh** | Processamento completo de todos os dados a cada execução (abordagem inicial do pipeline) |
| **SCD (Slowly Changing Dimensions)** | Técnica para gerenciar mudanças em dados dimensionais. Usamos **Type 1** (sobrescrever) para dimensões simples |
| **MERGE Operation** | Operação UPSERT (UPDATE + INSERT) do Delta Lake para processamento incremental dimensional |
| **Data Freshness** | Atualidade dos dados, medida em horas desde a última atualização. Meta: <26 horas |
| **Spot Instances** | Instâncias de computação com custo reduzido (até 70% mais baratas) para processamento noturno |

#### 3.1. Camadas do Pipeline (Medallion Architecture)

| Camada | Função Principal | Processo Chave |
| :--- | :--- | :--- |
| Bronze | Ingestão/Landing Zone. Contém dados brutos e imutáveis da fonte (CSV/Volume), após mínimas padronizações de schema e nome de coluna. | COPY INTO e Padronização de chaves (order_id). |
| Silver | Transformação/Limpeza. Onde a lógica de Unit Economics (UE) é aplicada. Dados são limpos, filtrados (delivery_status = 'DELIVERED') e as tabelas de fatos são unificadas via INNER JOIN. | Cálculo de UE (P&L Unitário). |
| Gold | Consumo/BI. Camada final com dados altamente estruturados e modelados em Star Schema para análise otimizada em ferramentas como o Power BI. | Modelagem Dimensional. |
| **Controle** | **Otimização/Governança**. Gerencia processamento incremental e monitoramento de custos. | **Controle de lotes e health checks** |

#### 3.2. Tabelas do Pipeline

| Camada | Tabela | Descrição |
| :--- | :--- | :--- |
| Bronze | `tbl_fact_orders_bronze` | Dados brutos de pedidos |
| Bronze | `tbl_fact_deliveries_bronze` | Dados brutos de entregas |
| Bronze | `tbl_fact_payments_bronze` | Dados brutos de pagamentos |
| Bronze | `tbl_dim_stores_bronze` | Dados brutos de lojas |
| Bronze | `tbl_dim_hubs_bronze` | Dados brutos de hubs |
| Silver | `tbl_fact_pedidos_silver` | Vista unificada com JOIN das tabelas bronze, filtro `delivery_status = 'DELIVERED'` e cálculos de UE |
| Gold | `tbl_fato_delivery_gold` | Tabela fato principal com métricas financeiras |
| Gold | `tbl_dim_store_gold` | Dimensão de lojas |
| Gold | `tbl_dim_time_gold` | Dimensão temporal gerada a partir dos pedidos |
| **Controle** | **`ctrl_incremental_processing`** | **Tabela de controle que registra últimos lotes processados** |
| **Controle** | **`log_processamento`** | **Logs de execução do pipeline para auditoria** |

#### 3.3. Modelagem Dimensional e Chaves

| Termo | Definição no Projeto | Contexto |
| :--- | :--- | :--- |
| Star Schema | Modelo de dados que usa uma Tabela Fato Central (tbl_fato_delivery_gold) ligada a várias Tabelas Dimensão (tbl_dim_store_gold, tbl_dim_time_gold) por chaves. Essencial para performance em BI. | Otimização de BI |
| Chave de Tempo (Order Date Key) | Coluna do tipo DATE na Tabela Fato que se liga à Dimensão Tempo. Foi criada usando a conversão explícita TO_TIMESTAMP('M/d/yyyy h:mm:ss a') para lidar com a string complexa de data/hora da fonte. | Conversão de Tipo |
| **Chave de Incrementalidade** | **Colunas `_loaded_date` e `_batch_id` adicionadas às tabelas Bronze para controle do processamento incremental** | **Otimização de Custo** |

#### 4. Estratégias de Otimização (Novo!)

| Termo | Definição | Benefício |
| :--- | :--- | :--- |
| **Processamento Incremental** | Estratégia onde apenas dados novos desde o último processamento são transformados | Redução de 80-90% no custo computacional |
| **Cluster Econômico** | Configuração de cluster com instâncias mais baratas (Standard_DS3_v2) e Spot Instances | Redução de 70% no custo de infraestrutura |
| **Agendamento Inteligente** | Execução do pipeline em horários de baixa demanda (ex: 2h AM) | Melhor utilização de recursos e custos reduzidos |
| **Health Check Automatizado** | Validação contínua da qualidade e performance do pipeline | Detecção proativa de problemas |
| **Validação Paralela** | Execução do novo pipeline em paralelo com o legado durante a transição | Migração segura sem risco de quebra |
| **ZORDER Otimizado** | Clustering físico de dados nas tabelas Gold por `(order_date_key, store_id)` | Performance otimizada para queries do Power BI |

#### 5. Métricas de Performance e Custo

| Métrica | Definição | Meta |
| :--- | :--- | :--- |
| **Custo Mensal Pipeline** | Custo total de processamento no Databricks | < $3.00/mês |
| **Tempo de Execução** | Tempo total do pipeline from Bronze to Gold | < 5 minutos |
| **Data Freshness** | Tempo entre geração do dado e disponibilidade no BI | < 4 horas |
| **Taxa de Processamento** | Registros processados por segundo | > 1.000 records/sec |
| **ROI da Otimização** | Retorno do investimento em desenvolvimento incremental | > 500% no primeiro ano |

#### 6. Fases de Implementação

| Fase | Objetivo | Entregáveis |
| :--- | :--- | :--- |
| **Fase 0** | Pipeline Completo Full Refresh | Notebook 01_ETL_And_Quality |
| **Fase 1** | Estabilização e Validação | Dashboards Power BI + Treinamento |
| **Fase 2** | Bronze Incremental | Notebook 02_Incremental_Optimization |
| **Fase 3** | Silver Incremental | Processamento diário < 3min |
| **Fase 4** | Otimização Final | Custo mensal < $1.00 |

---

**Status do Glossário**: `ATUALIZADO`  
**Última Revisão**: `19/11/2025`  
**Próxima Revisão**: 